In [1]:
import sys
from collections import defaultdict

import pandas as pd
# import matplotlib.pyplot as plt
# %matplotlib inline
import sqlite3 as sql

# Here's What We're Going To Do

## 1. Subset Data by Category
    a. See affect on businesses, number of reviews, Users
    b. See affect on Tips, Photos
## 2. Subset Data by City

*Ideally, we can make some good abstractions in the process*

In [2]:
conn = sql.connect("dataset/database.sqlite")

In [3]:
df = pd.read_sql_query("select * from business", conn)
df.head()

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.8409,-79.3996,3.0,30,1
1,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,,16432 Old Statesville Rd,Huntersville,NC,28078,35.4371,-80.8437,4.0,42,1
2,--8LPVSo5i0Oo61X01sV9A,Valley Bone and Joint Specialists,,"3941 E Baseline Rd, Ste 102",Gilbert,AZ,85234,33.3795,-111.7280,4.5,3,1
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.1232,-115.1690,4.0,1389,1
4,--9QQLMTbFzLJ_oT-ON3Xw,Great Clips,,"1835 E Guadalupe Rd, Ste 106",Tempe,AZ,85283,33.3617,-111.9100,3.0,9,1


## 1. Subset Data by Category

In [4]:
df_cat = pd.read_sql_query("select * from category", conn)
df_cat.head()

,business_id,category
0,YDf95gJZaq05wvo7hTQbbQ,Shopping
1,YDf95gJZaq05wvo7hTQbbQ,Shopping Centers
2,mLwM-h2YhXl2NCgdS84_Bw,Food
3,mLwM-h2YhXl2NCgdS84_Bw,Soul Food
4,mLwM-h2YhXl2NCgdS84_Bw,Convenience Stores


In [5]:
categ = set(['Restaurants', 'Food', 'Bars'])

In [6]:
category_sets = defaultdict(set)
for row in df_cat.itertuples():
    cat = getattr(row, 'category')
    if cat in categ:
        bus_id = getattr(row, 'business_id')
        category_sets[cat].add(bus_id)

In [7]:
business_subset = set()
business_subset.update(category_sets['Restaurants'])
business_subset.update(category_sets['Food'])
business_subset.update(category_sets['Bars'])
print("{:<35} : {:>7}".format("Number of Businesses in our subset:", len(business_subset)))
print("{:<35} : {:>7}".format("Total Number of Businesses", df.shape[0]))

Number of Businesses in our subset: :   67882
Total Number of Businesses          :  156639


In [8]:
len(business_subset - category_sets['Bars'])

58014

In [9]:
business_subset_list = list(business_subset)

In [10]:
df_reviews = pd.read_sql_query("select user_id, business_id from review", conn)
df_reviews.head()

,user_id,business_id
0,gVmUR8rqUFdbSeZbsg6z_w,Ue6-WhXvI-_1xUIuapl0zQ
1,Y6qylbHq8QJmaCRSlKdIog,Ae4ABFarGMaI5lk1i98A0w
2,SnXZkRN9Yf060pNTk1HMDg,lKq4Qsz13FDcAVgp49uukQ
3,VcmSgvslHAhqWoEn16wjjw,6nKR80xEGHYf2UxAe_Cu_g
4,NKF9v-r0jd1p0JVi9h2T1w,Z_mJYg3vi8cPZHa1J4BALw


df_reviews = df_reviews[['user_id', 'business_id']]

In [11]:
users_subset = set()
num_reviews_subset = 0
for row in df_reviews.itertuples():
    bus_id = getattr(row, 'business_id')
    if bus_id in business_subset:
        users_subset.add(getattr(row, 'user_id'))
        num_reviews_subset += 1
    

In [12]:
num_users_total = len(set(df_reviews.user_id))
print("{:<30} : {:>7}".format("Number of Users in our subset:", len(users_subset)))
print("{:<30} : {:>7}".format("Total Number of Users", num_users_total))

Number of Users in our subset: :  869207
Total Number of Users          : 1183362


In [13]:
print("{:<35} : {:>7}".format("Number of Reviews in our subset", num_reviews_subset))
print("{:<35} : {:>7}".format("Total Number of Reviews", df_reviews.shape[0]))

Number of Reviews in our subset     : 3306658
Total Number of Reviews             : 4736897


In [14]:
df_cat.head()

,business_id,category
0,YDf95gJZaq05wvo7hTQbbQ,Shopping
1,YDf95gJZaq05wvo7hTQbbQ,Shopping Centers
2,mLwM-h2YhXl2NCgdS84_Bw,Food
3,mLwM-h2YhXl2NCgdS84_Bw,Soul Food
4,mLwM-h2YhXl2NCgdS84_Bw,Convenience Stores


## Results from the Category subsetting

In [15]:
print("{:<35} : {:>7}".format("Number of Businesses in our subset:", len(business_subset)))
print("{:<35} : {:>7}".format("Total Number of Businesses", df.shape[0]))
print()
print("{:<35} : {:>7}".format("Number of Users in our subset:", len(users_subset)))
print("{:<35} : {:>7}".format("Total Number of Users", num_users_total))
print()
print("{:<35} : {:>7}".format("Number of Reviews in our subset", num_reviews_subset))
print("{:<35} : {:>7}".format("Total Number of Reviews", df_reviews.shape[0]))

Number of Businesses in our subset: :   67882
Total Number of Businesses          :  156639

Number of Users in our subset:      :  869207
Total Number of Users               : 1183362

Number of Reviews in our subset     : 3306658
Total Number of Reviews             : 4736897


# 2. Subset Data by City

In [16]:
city_lat_long_dict = {
    'las_vegas' : (36.1699, -115.1398),
    'phoenix' : (33.4484, -112.0740),
    'champaign' : (40.1164, -88.2434),
    'madison' : (43.0731, -89.4012),
    'cleveland' : (41.4993, -81.6944),
    'pittsburgh' : (40.4406, -79.9959),
    'toronto' : (43.6532, -79.3832),
    'montreal' : (45.5017, -73.5673),
    'charlotte' : (35.2271, -80.8431),
    'stuttgart' : (48.7758, 9.1829),
    'endinburgh' : (55.9533, -3.1883)
}
len(city_lat_long_dict)

11

#### So we have a list of businesses but now need the dataframe with those businesses. At the very least, a list of tuples with business_id, lat, long

In [17]:
len(business_subset)

67882

In [18]:
df_business_subset = df[df['id'].isin(business_subset)]
df_business_subset.head()

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open
0,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.8409,-79.3996,3.0,30,1
1,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,,16432 Old Statesville Rd,Huntersville,NC,28078,35.4371,-80.8437,4.0,42,1
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.1232,-115.1690,4.0,1389,1
7,--cjBEbXMI2obtaRHNSFrA,Howl at the Moon,Downtown,125 7th St,Pittsburgh,PA,15222,40.4439,-80.0002,3.0,44,1
8,--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Dilworth,1408 E Blvd,Charlotte,NC,28203,35.1999,-80.8448,4.0,299,1


In [19]:
df_business_subset.loc[:,'metropolitan_area'] = None
df_business_subset.loc[:,'distance_from_metropolitan_center'] = None

/Users/David/anaconda/envs/3.6default/lib/python3.6/site-packages/pandas/core/indexing.py:357: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
/Users/David/anaconda/envs/3.6default/lib/python3.6/site-packages/pandas/core/indexing.py:537: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [22]:
for row in df_business_subset[['latitude', 'longitude']].itertuples():
    lat = getattr(row, 'latitude')
    long = getattr(row, 'longitude')
    closest_city = None
    closest_dist = 0
    for city, (true_lat, true_long) in city_lat_long_dict.items():
        dist = (true_lat - lat)**2 + (true_long - long)**2
        if dist < closest_dist or not closest_city:
            closest_city = city
            closest_dist = dist
    df_business_subset.at[row[0], 'metropolitan_area'] = closest_city
    df_business_subset.at[row[0], 'distance_from_metropolitan_center'] = closest_dist

In [25]:
df_business_subset.head()

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,metropolitan_area,distance_from_metropolitan_center
0,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.8409,-79.3996,3.0,30,1,toronto,0.0355002
1,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,,16432 Old Statesville Rd,Huntersville,NC,28078,35.4371,-80.8437,4.0,42,1,charlotte,0.0441004
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.1232,-115.1690,4.0,1389,1,las_vegas,0.00303353
7,--cjBEbXMI2obtaRHNSFrA,Howl at the Moon,Downtown,125 7th St,Pittsburgh,PA,15222,40.4439,-80.0002,3.0,44,1,pittsburgh,2.938e-05
8,--cZ6Hhc9F7VkKXxHMVZSQ,Pio Pio,Dilworth,1408 E Blvd,Charlotte,NC,28203,35.1999,-80.8448,4.0,299,1,charlotte,0.00074273


In [31]:
df_business_subset.shape

(67882, 14)

## See what happens when we only keep businesses within .1 square lat/longs

In [32]:
# Remember: we are in lat-long squared...There could be weird conversions

df_business_subset2 = df_business_subset[df_business_subset.distance_from_metropolitan_center < .1]

In [34]:
users_subset_with_dist = set()
num_reviews_subset_with_dist = 0
for row in df_reviews.itertuples():
    bus_id = getattr(row, 'business_id')
    if bus_id in business_subset:
        users_subset_with_dist.add(getattr(row, 'user_id'))
        num_reviews_subset_with_dist += 1
    

In [35]:
num_users_total = len(set(df_reviews.user_id))
print("{:<30} : {:>7}".format("Number of Users in our subset:", len(users_subset_with_dist)))
print("{:<30} : {:>7}".format("Total Number of Users", num_users_total))
print()
print("{:<35} : {:>7}".format("Number of Reviews in our subset", num_reviews_subset_with_dist))
print("{:<35} : {:>7}".format("Total Number of Reviews", df_reviews.shape[0]))

Number of Users in our subset: :  869207
Total Number of Users          : 1183362

Number of Reviews in our subset     : 3306658
Total Number of Reviews             : 4736897


In [ ]:
df_total.to_sql('checkin_total', conn, index = False)

In [42]:
pd.read_sql_query("SELECT * FROM sqlite_master where type='table'", conn).at[13, 'sql']

'CREATE TABLE "business_subset" (\n"index" INTEGER,\n  "id" TEXT,\n  "name" TEXT,\n  "neighborhood" TEXT,\n  "address" TEXT,\n  "city" TEXT,\n  "state" TEXT,\n  "postal_code" TEXT,\n  "latitude" REAL,\n  "longitude" REAL,\n  "stars" REAL,\n  "review_count" INTEGER,\n  "is_open" INTEGER,\n  "metropolitan_area" TEXT,\n  "distance_from_metropolitan_center" REAL\n)'

In [38]:
temp = pd.read_sql_query("select * from business2", conn)

In [39]:
temp.head()

,id,name,neighborhood,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,tip_count,photo_count
0,--6MefnULPED_I942VcFNA,John's Chinese BBQ Restaurant,,"328 Highway 7 E, Chalmers Gate 11, Unit 10",Richmond Hill,ON,L4B 3P7,43.8409,-79.3996,3.0,30,1,12.0,1.0
1,--7zmmkVg-IMGaXbuVd0SQ,Primal Brewery,,16432 Old Statesville Rd,Huntersville,NC,28078,35.4371,-80.8437,4.0,42,1,10.0,NaN
2,--8LPVSo5i0Oo61X01sV9A,Valley Bone and Joint Specialists,,"3941 E Baseline Rd, Ste 102",Gilbert,AZ,85234,33.3795,-111.7280,4.5,3,1,NaN,NaN
3,--9e1ONYQuAa-CB_Rrw7Tw,Delmonico Steakhouse,The Strip,3355 Las Vegas Blvd S,Las Vegas,NV,89109,36.1232,-115.1690,4.0,1389,1,144.0,127.0
4,--9QQLMTbFzLJ_oT-ON3Xw,Great Clips,,"1835 E Guadalupe Rd, Ste 106",Tempe,AZ,85283,33.3617,-111.9100,3.0,9,1,2.0,NaN


In [40]:
df_business_subset.to_sql('business_subset', conn)

In [43]:
df_business_subset2.to_sql('business_subset_with_distance', conn)